In [ ]:
# Import all required libraries
import torch
import tensorflow as tf
from models.gpt_generator import GPTGenerator
from models.lstm_generator import LSTMModel, TextTokenizer
from utils.preprocess import clean_text, create_sequences
import numpy as np
import time
import nltk

In [ ]:
# Ensure NLTK data is downloaded
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

In [ ]:
# ----------------------------
# 1. Environment Verification
# ----------------------------
print("="*50)
print("GPU/CPU Verification")
print("="*50)

# PyTorch verification
torch_available = torch.cuda.is_available()
torch_device = torch.device('cuda' if torch_available else 'cpu')
print(f"\nPyTorch:")
print(f"• CUDA Available: {torch_available}")
if torch_available:
    print(f"• Device: {torch.cuda.get_device_name(0)}")
    print(f"• CUDA Version: {torch.version.cuda}")

# TensorFlow verification
tf_devices = tf.config.list_physical_devices('GPU')
print(f"\nTensorFlow:")
print(f"• GPUs Available: {len(tf_devices)}")
if tf_devices:
    print(f"• GPU Details: {tf_devices}")

print("\n" + "="*50 + "\n")

: 

In [ ]:
# ----------------------------
# 2. GPT-2 Text Generation
# ----------------------------
print("="*50)
print("GPT-2 Text Generation")
print("="*50)

# Initialize generator
gpt = GPTGenerator()

# Generate text
prompt = "Artificial intelligence will"
start_time = time.time()
generated_text = gpt.generate_text(
    prompt,
    max_length=150,
    temperature=0.7,
    top_k=50
)
gpt_time = time.time() - start_time

print(f"\nPrompt: '{prompt}'")
print(f"Generation Time: {gpt_time:.2f} seconds")
print("\nGenerated Text:")
print("-"*50)
print(generated_text)
print("-"*50)

print("\n" + "="*50 + "\n")

In [ ]:
# ----------------------------
# 3. LSTM Text Generation
# ----------------------------
print("="*50)
print("LSTM Text Generation")
print("="*50)

# Load and preprocess data
with open('data/sample_texts.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

cleaned_text = clean_text(raw_text)
tokenizer = TextTokenizer(cleaned_text)

# Prepare training data
sequences, next_chars = create_sequences(cleaned_text, seq_length=50)
X = np.array([[tokenizer.char_to_idx[c] for c in seq] for seq in sequences])
y = tf.keras.utils.to_categorical(
    [tokenizer.char_to_idx[c] for c in next_chars],
    num_classes=tokenizer.vocab_size
)

# Initialize and train model
lstm = LSTMModel(tokenizer, seq_length=50)

print("\nTraining LSTM model (3 epochs demo)...")
train_start = time.time()
lstm.train(X, y, epochs=3, batch_size=128)
train_time = time.time() - train_start

# Generate text
seed = "The future of AI is"
gen_start = time.time()
lstm_output = lstm.generate_text(
    seed,
    length=200,
    temperature=0.6
)
gen_time = time.time() - gen_start

print(f"\nTraining Time: {train_time:.2f} seconds")
print(f"Seed Text: '{seed}'")
print(f"Generation Time: {gen_time:.2f} seconds")
print("\nGenerated Text:")
print("-"*50)
print(lstm_output)
print("-"*50)

print("\n" + "="*50 + "\n")

In [ ]:
# ----------------------------
# 4. Performance Comparison
# ----------------------------
print("="*50)
print("Performance Summary")
print("="*50)
print(f"GPT-2 Generation Time: {gpt_time:.2f}s")
print(f"LSTM Training Time (3 epochs): {train_time:.2f}s")
print(f"LSTM Generation Time: {gen_time:.2f}s")

# Cleanup
torch.cuda.empty_cache()
tf.keras.backend.clear_session()

print("\nGPU memory cleared. Execution complete!")
print("="*50)